# Load txt to parquet -  ingestion phase

1) Dataframe schema definition<BR> 
2) Read data from azure Row folder to landing zone<BR>
3) Create Audit Log ??

## 1.Setup Folder and Files

### 1.1 List files in relevent storage

In [1]:
import warnings
warnings.simplefilter(action='ignore') #, category=FutureWarning

In [2]:
import os
cwd = os.getcwd()
print(cwd)

c:\BI\GitHub\LearningSparkV2-master\SuppliesML\02-Data-Engineering\pyspark\02-LoadData




This is an example of how to list things you need to use the software and how to install them.
* Azure blob storage indigo 
  ```sh
 display(dbutils.fs.ls(srcDataDirRoot))
  ```

In [3]:
MOUNTPOINT = "c:\BI\GitHub\LearningSparkV2-master\SuppliesML"
SourcePath = os.path.join(MOUNTPOINT, "Data","Dev")
RawPath    = os.path.join(SourcePath,"Raw")
ResearchPath = os.path.join(SourcePath,"Research")
LandingPath = os.path.join(SourcePath, "Landing")
LoadingPath = os.path.join(SourcePath,  "Loading")
certified_pit = os.path.join(SourcePath,"certified-pit")

In [4]:
# Define source and destination directories
srcDataDirRoot = RawPath #Root dir for source data
destDataDirRoot = LandingPath #Root dir for consumable data

### 1.2 Define Schema for the file that loaded

In [10]:
# Required for StructField, StringType, IntegerType, etc.
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, TimestampType


# # Space bi inc- sensor data 
ProductionRunSchema = StructType([
  StructField("Product" , StringType() ,True),
  StructField("Machine" , StringType() ,True),
  StructField("FOLDER_PATH" , StringType() ,True),
  StructField("Product_category" , StringType() ,True),
  StructField("Product_eng_name" , StringType() ,True),
  StructField("Series" , StringType() ,True),
  StructField("Product_name_win" , StringType() ,True),
  StructField("Size_Flag" , StringType() ,True),
  StructField("Parameter_Name" , StringType() ,True),
  StructField("Batch" , StringType() ,True),
  StructField("SAMPLE_ID" , FloatType() ,True),
  StructField("SAMPLE_Date" , TimestampType() ,True),
  StructField("SK_Sample_Date" , IntegerType() ,True),
  StructField("Parameter_Critical_Flag" , StringType() ,True),
  StructField("Is_Sample_Deleted_Flg" , StringType() ,True),
  StructField("SAMPLE_Mean" , FloatType() ,True),
  StructField("SAMPLE_stdev" , FloatType() ,True),
  StructField("SAMPLE_Minimum" , FloatType() ,True),
  StructField("SAMPLE_Maximum" , FloatType() ,True),
  StructField("SAMPLE_Median" , FloatType() ,True),
  StructField("Spec_target" , FloatType() ,True),
  StructField("SAMPLE_Size" , FloatType() ,True),
  StructField("LSL" , FloatType() ,True),
  StructField("USL" , FloatType() ,True),
  StructField("SL_enabled" , StringType() ,True),
  StructField("CH_ID" , FloatType() ,True),
  StructField("ETL_DATE" , TimestampType() ,True)
])


# windigo production line
New_Era_Run_Details_Schema = StructType([
StructField("Run_Number" , StringType() ,True),
StructField("Blanket_Serial_Number" , StringType() ,True),  
StructField("Blanket_SEQ_NR" , StringType() ,True),  
StructField("Plant_Id" , IntegerType() ,True),
StructField("Quality_Status_Id" , IntegerType() ,True),
StructField("Body_ID" , StringType() ,True),
StructField("CSL_ID" , StringType() ,True),
StructField("BLK_Quality_Status_Name" , StringType() ,True),
StructField("BLK_Quality_Status_Flag" , IntegerType() ,True),
StructField("Blanket_Legacy_Part_Nr" , StringType() ,True),
StructField("Product_Engineering_Name" , StringType() ,True),
StructField("Source_System_Modified_DateTime" , TimestampType() ,True)
])


# # Customer data - lifespan
Blanket_lifespan_installed_base_Schema = StructType([
#StructField("Fact_PIP_IMPACT_RowID", IntegerType() ,True),
StructField("Blanket_Impact_RowID", StringType() ,True),
StructField("Press_Serial_Number", StringType() ,True),
StructField("BLANKETS_ID", StringType() ,True),
StructField("Replacement_DateTime", StringType() ,True),
StructField("End_User_Code", StringType() ,True),
StructField("Domain", StringType() ,True),
StructField("ROR", StringType() ,True),
StructField("Consumable_Type", StringType() ,True),
StructField("Optimized_Lifespan", IntegerType() ,True),
StructField("Is_Last_Replacement", StringType() ,True),
StructField("Is_Lifespan_Official", StringType() ,True),
StructField("Consumable_Maturity", StringType() ,True),
StructField("DOA_Count", IntegerType() ,True),
StructField("DOP_Count", IntegerType() ,True),
StructField("RowID", IntegerType() ,True),
StructField("Changed_Date_Time", StringType() ,True),
StructField("Replacement_Monthly_Date_Id", IntegerType() ,True),
StructField("ETL_Date", StringType() ,True),
#StructField("Press_Classification", StringType() ,True),
#StructField("Lifespan_Guidelines", DoubleType() ,True),
StructField("Click_Charge", StringType() ,True),
StructField("Ownership", StringType() ,True),
StructField("Product_Number", StringType() ,True),
StructField("Description", StringType() ,True),
StructField("Product_Group", StringType() ,True),
StructField("Press_Group", StringType() ,True),
StructField("Family_type", StringType() ,True),
StructField("Series", StringType() ,True),
StructField("Press_Segment", StringType() ,True),
StructField("Current_SW_Version_ID", StringType() ,True),
StructField("Customer_Name", StringType() ,True),
StructField("Site_Region", StringType() ,True),
StructField("Site_Sub_Region", StringType() ,True),
StructField("Site_Country", StringType() ,True),
#StructField("ETL_Date" , TimestampType() ,True)
])



## 2.Create Spark Session

### 2.1 check warehouse Location

In [11]:
from os.path import abspath
import pyodbc
warehouse_location = abspath('..\..\..\supplies-warehouse')
print(warehouse_location)

c:\BI\GitHub\LearningSparkV2-master\SuppliesML\supplies-warehouse


In [12]:
from os.path import abspath
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
# warehouse_location points to the default location for managed databases and tables
#warehouse_location = abspath('..\..\..\supplies-warehouse')
warehouse_location = abspath('..\..\..\hive_supplies-warehouse')

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example")\
.config("spark.hadoop.hive.metastore.warehouse.dir", warehouse_location)\
.config('spark.driver.extraClassPath', 'C:\Program Files\Microsoft JDBC Driver 10.2 for SQL Server\sqljdbc_10.2\enu\mssql-jdbc-10.2.1.jre8.jar') \
.config('spark.executor.extraClassPath', 'C:\Program Files\Microsoft JDBC Driver 10.2 for SQL Server\sqljdbc_10.2\enu\mssql-jdbc-10.2.1.jre8.jar') \
.config("spark.sql.catalogImplementation","hive") \
.enableHiveSupport() \
.getOrCreate() 


## 3. set data source Connections

### 3.1 Set odbc Connections > user + Password

In [8]:
#Driver={SQL Server Native Client 11.0};Server=myServerAddress;Database=myDataBase;Uid=myUsername;Pwd=myPassword;

sql_native_space = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=gvs72076.inc.hpicorp.net,2048;"
                        "Database=faomrp;"
                        "Uid=SPACE_BI_IDS;"
                        "Pwd=SPC-IDS!pswd;")
                  
# > windows trusted authication  - when we user password please assign it to "no"
# conn_GABI.close()
                        

In [ ]:
server = 'gvs72076.inc.hpicorp.net,2048' 
database = 'faomrp' 
username = 'SPACE_BI_IDS' 
password = 'SPC-IDS!pswd' 
odbc_SPACE_BI = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
print(odbc_SPACE_BI,database)

# server = '106775-2s-ag1.corp.hpicloud.net,2048' 
# database = 'GABI-I' 
# username = 'GABI_STG_RW_1' 
# password = 'change_pw_first_login2020$' 
# odbc_SPACE_BI = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
# print(odbc_SPACE_BI,database)

server = 'gvs72069.inc.hpicorp.net,2048' 
database = 'GABI-P' 
username = 'GABI_STG_RW_1' 
password = 'fT7_E*4U6k_m' 
odbc_GABI_P = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
print(odbc_SPACE_BI,database)


server = 'WIN-OA97GH5SPHI.CORP.HPICLOUD.NET,1433' # 172.20.206.100
database = 'Hubble_DWH_ITG' 
username = 'Tableau_Itg' 
password = 'Tab_itg4' 
odbc_Hubble_ITG = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
print(odbc_Hubble_ITG,database)



<pyodbc.Connection object at 0x000001F6E8EBA800> faomrp
<pyodbc.Connection object at 0x000001F6E8EBA800> GABI-P


ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'Tableau_Itg'. Reason: The account is disabled. (18470) (SQLDriverConnect); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'Tableau_Itg'. Reason: The account is disabled. (18470)")

### 3.2 Set odbc Connections > user authntication

In [ ]:
import pyodbc
Driver="{ODBC Driver 17 for SQL Server}"
server="172.20.206.100,1433"
database="HUBBLE_DWH_ITG"
trusted_connection="yes"
odbc_Hubble_ITG = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';Database='+database+';trusted_connection='+trusted_connection)
print(odbc_Hubble_ITG)

In [ ]:
#Driver={SQL Server Native Client 11.0};Server=myServerAddress;Database=myDataBase;Uid=myUsername;Pwd=myPassword;
import pyodbc
sql_native_Hubble_ITG = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=172.20.206.100,1433;"
                        "Database=HUBBLE_DWH_ITG;"
                        "Uid=Tableau_Itg;"
                        "Pwd=Tab_itg4;")
                  
# > windows trusted authication  - when we user password please assign it to "no"
# conn_GABI.close()

### 3.3 Set odbc Connections > user + passward

sqlsUrl = 'jdbc:sqlserver://172.20.206.100:1433;encrypt=true;databaseName=HUBBLE_DWH_ITG;integratedSecurity=true;trustServerCertificate=true;'

>> there is no integrated security 

### 3.4 set jdbc Connections

In [ ]:
#sqlsUrl = 'jdbc:sqlserver://gvs72069.inc.hpicorp.net:2048;encrypt=true;database=GABI-P;user=GABI_STG_RW_1;password=fT7_E*4U6k_m;trustServerCertificate=true;'
JdbcsqlsUrl_space = 'jdbc:sqlserver://gvs72076.inc.hpicorp.net:2048;encrypt=true;database=faomrp;user=SPACE_BI_IDS;password=SPC-IDS!pswd;trustServerCertificate=true;'

In [ ]:
JdbcsqlsUrl_NewEra = 'jdbc:sqlserver://gvs72069.inc.hpicorp.net:2048;encrypt=true;database=GABI-P;user=GABI_STG_RW_1;password=fT7_E*4U6k_m;trustServerCertificate=true;'

In [ ]:
JdbcsqlsUrl_blanketslifespan = 'jdbc:sqlserver://WIN-OA97GH5SPHI.CORP.HPICLOUD.NET:1433;encrypt=true;database=HUBBLE_DWH_ITG;user=Tableau_Itg;password=Tab_itg4;trustServerCertificate=true;'

## 5. List of queires

## 5.1 Load with JDBC

#### 5.1.1 Load from New Era

In [ ]:
import time
from datetime import datetime, timedelta,date
# Build Dynamic Query per yearmonth filter
def Loaddata(YearMonth,StartYear,querypath):
    
    #YearMonth ="201903"
    Year=YearMonth[0:4]
    with open(querypath) as f: query_file = f.read()
    sql_string = query_file.replace("{YearMonth}",str(YearMonth)).replace("{StartYear}",str(StartYear))
    dynamic_sql = f"""{sql_string}"""
    return dynamic_sql


# Read Query from Source >  Return DATAFRAME per yearmonth filter
def sprak_read_df(Query,schema):
    
    startTime = time.time()
    spark_read_df = spark.read.format('jdbc') \
    .option('url',JdbcsqlsUrl_NewEra) \
    .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver')\
    .option('query', Query) \
    .schema(schema) \
    .load()
    executionTime = (time.time() - startTime)
    #print('Execution time in seconds: ' + str(round(executionTime,2)))
    return spark_read_df


def WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,YearMonth):
    Dataframe.coalesce(8).write.parquet(LandingPath + Table_Folder +str(Year)+"/"+str(YearMonth),mode="overwrite") 
    executionTime = (time.time() - startTime)

min_query_date= '2020-11-01'  
StartYear = "2018"
Zone = LandingPath
Table_Folder = "/New_Era/New_era_details/"
Table = "NewEra"
QueryPath = "IngestQueries/NewEra.sql"
startPipelineTime =  time.time()

#for beg in pd.date_range(min_query_date,date.today().strftime("%Y-%m-01"), freq='MS'): # date.today() date.today().strftime("%Y-%m-01")
for beg in pd.date_range('2023-01-01','2023-01-31', freq='MS'): # date.today()
    startTime = time.time()
    Yearmonth = beg.strftime("%Y%m")
    Year=Yearmonth[0:4]                                     
    dynamic_sql = Loaddata(Yearmonth,StartYear,QueryPath)   
    
    Dataframe = sprak_read_df(dynamic_sql,New_Era_Run_Details_Schema)
    WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,Yearmonth)
    
    executionTime = (time.time() - startTime)
    print(f'Execution time in seconds for: Table '+Table+' > YearMonth: '+ Yearmonth +"  is: "+ str(round(executionTime,2))+ "   numberofrow:  " + str(Dataframe.count()))
executionTime = (time.time() - startPipelineTime)
print(f'total Execution time in seconds for: Table '+Table+' > all yearmonth is: '+ str(round(executionTime,2)))   
    

Execution time in seconds for: Table NewEra > YearMonth: 202301  is: 171.99   numberofrow:  75404
total Execution time in seconds for: Table NewEra > all yearmonth is: 189.48


### 5.1.2.1 read & write & Test Count / Show() all NEW ERA yearmonth file to one parquet file

In [ ]:
New_era_details_dfs = spark.read.format("parquet").load(LandingPath + "/New_Era/New_era_details/*/*") # try to read empty

In [ ]:
import time
startTime = time.time()
New_era_details_dfs.coalesce(1).write.parquet(LandingPath + "/New_Era/New_era_history",mode="overwrite") #e")#.saveAsTable("ld_blanket_lifespan_run_extended")# = spark.read.format("parquet").load(SourcePath + '/test')
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(round(executionTime,2)))


Execution time in seconds: 11.08


#### 5.1.2 Load from SpaceBI > production run jdbc

```python
import os
os.chdir(r'C:\BI\GitHub\LearningSparkV2-master\SuppliesML\02-Data-Engineering\pyspark\02-LoadData')
pwd()
```

In [ ]:
#with open("IngestQueries\ProductionLine.sql") as f: query_file = f.read()

In [24]:
import time
from datetime import datetime, timedelta,date
import calendar
from dateutil.relativedelta import relativedelta

# Build Dynamic Query per yearmonth filter
def Loaddata(first_day_of_month:datetime, last_day_of_month:datetime,querypath:str) -> str:
    """ Get the Start date and the end date of the desired query range to filter

    Parameters
    ----------
    first_day_of_month : datetime
        start date of main fact to filter
    last_day_of_month : datetime
        end date of main fact to filter
    querypath: str
    
    log created
    ----------
    create dynamic_sql txt file to review the sql script 

    Returns
    -------
    str : query string the change dynmicly by parameters
    """
    with open(querypath) as f: query_file = f.read()
    sql_string = query_file.replace("{first_day_of_month}","'"+str(first_day_of_month)+"'")\
        .replace("{last_day_of_month}","'"+str(last_day_of_month)+"'").replace("{FolderPath}",str( "'"+FolderPath+"'"))
    dynamic_sql = f"""{sql_string}"""
    file = open("dynamic_sql.txt", "w")
    a = file.write(dynamic_sql)
    file.close()
    return dynamic_sql

def sprak_read_df(query,schema,conn):
    """ Read Query from Source >  Return DATAFRAME per yearmonth filter

    Parameters
    ----------
    query : str
        dynamic sql 
    schema : str
        table schema
    conn: connection to db

    Returns
    -------
    dataframe : return spark dataframe
    """

    startTime = time.time()
    spark_read_df = spark.read.format('jdbc') \
    .option('url',conn) \
    .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver')\
    .option('query', query) \
    .schema(schema) \
    .load()
    executionTime = (time.time() - startTime)
    return spark_read_df


def pandas_read_df(dynamic_sql,sql_native_space):
    fmt='%Y%m%d %H:%M:%S'             
    pandas_read_pdf = pd.read_sql_query(sql = dynamic_sql,\
                                        con = sql_native_space,#\
                                        #parse_dates={'SAMPLE_Date':fmt,'ETL_DATE':fmt}\
                                       )                                  
    return pandas_read_pdf

def WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,Yearmonth):
    spark_read_df=spark.createDataFrame(Dataframe,ProductionRunSchema) #ProductionRunSchema
    spark_read_df.coalesce(1).write.parquet(LandingPath + Table_Folder +str(Year)+"/"+str(Yearmonth),mode="overwrite") 
    executionTime = (time.time() - startTime)
#

def start_end_month(date:datetime):
    
    first_day = beg + relativedelta(day=1)
    last_day = (beg.replace(day=1) + timedelta(days=32)).replace(day=1)
    return first_day,last_day

ProductList = {}
ProductList['Gemini3'] = {'StartYearMonth': '2020-10-01', 'FullPath': 'New Era+AFL+WSM-> Gemini 3','FolderOutput':'Gemini', 'ProductName': 'GEMINI3'}
ProductList['RotemSP'] = {'StartYearMonth': '2020-11-01', 'FullPath': 'New Era+AFL+WSM-> NewEra-SpeedUp-> Rotem SP','FolderOutput':'RotemSP', 'ProductName': 'ROTEM_SP'}
ProductList['IrisPlusSP'] = {'StartYearMonth': '2020-10-01', 'FullPath': 'New Era+AFL+WSM-> NewEra-SpeedUp-> Iris Plus SP','FolderOutput':'IrisPlusSP', 'ProductName': 'IRIS PLUS_SP'}
ProductList['Timna2'] = {'StartYearMonth': '2020-10-01', 'FullPath': 'New Era+AFL+WSM-> Timna 2','FolderOutput':'Timna2', 'ProductName': 'TIMNA 2'}


### Production run queries variables
## **************************************
# find Min Date To load Data
querypath = "IngestQueries/ProductionLine.sql"
min_date = '2019-01-01'
StartYear = "2018"
FolderPath = ProductList['Timna2']['FullPath']
ProductName = ProductList['Timna2']['ProductName']
with open(querypath) as f: query_file = f.read()
sql_string = query_file.replace("{StartYear}",str(StartYear)).replace("{FolderPath}",str("'"+FolderPath+"'")).replace("{FolderPath}",str( "'"+FolderPath+"'"))
dynamic_sql = f"""{sql_string}"""
#print(str("'"+FolderPath+"'"))

#min_query_date_pdf = pd.read_sql_query(sql = dynamic_sql,  con = sql_native_space)  
#start_date = min_query_date_pdf['MinYearMonth'][0].strftime("%Y-%m-01")
#min_query_date = str(max(min_date,start_date))
## **************************************

min_query_date= '2022-12-01'   
Zone = LandingPath
product = "IrisPlusSP"   # Iris 6234  '2022-11-01'  # rptem Count before inner 2460
Table_Folder = "/ProductionRun/ProductionRun_hist/"+product+"/"
#Table_Folder = "/New_Era/New_era_details/"
Table = "ProductionRun"
QueryPath = "QaQueries\ProductionLine_test.sql"
#Test > \QaQueries\ProductionLine_test.sql   / prod "IngestQueries/ProductionLine.sql"
startPipelineTime =  time.time()

for beg in pd.date_range(min_query_date,'2022-12-31' , freq='MS'): # date.today() date.today().strftime("%Y-%m-01")
    first_day_of_month, last_day_of_month = start_end_month(beg)
    # startTime = time.time()
    # Yearmonth = beg.strftime("%Y%m")
    # Year=Yearmonth[0:4]                                     
    dynamic_sql = Loaddata(first_day_of_month,last_day_of_month,QueryPath)   
    print(dynamic_sql)
    print('finish dynmic',Yearmonth)
    Dataframe = pandas_read_df(dynamic_sql,sql_native_space)
    print('finish READ QUERY')
    Dataframe.to_csv("test3.csv",index= False)
    #print(Dataframe.columns)
    WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,Yearmonth)
    
    executionTime = (time.time() - startTime)
    print(f'Execution time in seconds for: Table '+Table+' > YearMonth: '+ Yearmonth +"  is: "+ str(round(executionTime,2))+ "   numberofrow:  " + str(len(Dataframe.index)))

executionTime = (time.time() - startPipelineTime)
print(f'{datetime.now()} +   total Execution time in seconds for: Table '+Table+' > all yearmonth is: '+ str(round(executionTime,2)))   





with Folders_Id_list AS (
--******

-- Create a table that contain all the relevent Blankets products for the ML analysis
select  6743	 AS [FOLDER_ID],'New Era+AFL+WSM'     as [Folder Name], 0 AS Parent, 'BLANKETS' AS product, 8 As LDS_ID UNION
select  9908     AS [FOLDER_ID],'New Era+AFL+WSM-> Gemini 3'     as [Folder Name],  1 AS Parent, 'GEMINI3' AS product, 8 As LDS_ID UNION
select  9907     AS [FOLDER_ID],'New Era+AFL+WSM-> Timna 2'     as [Folder Name],  1 AS Parent, 'TIMNA 2' AS product, 8 As LDS_ID UNION
select  11163     AS [FOLDER_ID],'New Era+AFL+WSM-> NewEra-SpeedUp'     as [Folder Name],  0 AS Parent, 'BLANKETS' AS product, 8 As LDS_ID UNION
select  11165     AS [FOLDER_ID],'New Era+AFL+WSM-> NewEra-SpeedUp-> Iris Plus SP'     as [Folder Name],  1 AS Parent, 'IRIS PLUS_SP' AS product, 8 As LDS_ID UNION
select  11168     AS [FOLDER_ID],'New Era+AFL+WSM-> NewEra-SpeedUp-> Rotem SP'     as [Folder Name],  1 AS Parent, 'ROTEM_SP' AS product, 8 As LDS_ID 
)																			

NameError: name 'Yearmonth' is not defined

In [8]:
queryPath = ""
#print(queryPath)
query_file = ""
#print(a2)
sql_string = ""
#print(a3)

print(f"xxxx > {sql_string}  yyy")
querypath = "QaQueries\ProductionLine_test copy.txt"
with open(querypath) as f: query_file = f.read()
sql_string = query_file.replace("{first_day_of_month}",str(first_day_of_month)).replace("{last_day_of_month}",str(last_day_of_month)).replace("{FolderPath}",str( "'"+FolderPath+"'"))
print(sql_string)

xxxx >   yyy


NameError: name 'first_day_of_month' is not defined

In [123]:
pwd

'c:\\BI\\GitHub\\LearningSparkV2-master\\SuppliesML\\02-Data-Engineering\\pyspark\\02-LoadData'

In [16]:
import calendar
from datetime import datetime
from dateutil.relativedelta import relativedelta

year =2022
month = 12
sample_date =  datetime(year, month, 1)
first_day_of_month = sample_date + relativedelta(day=1)
first_day_of_month = sample_date + relativedelta(day=31)
print(first_day, last_day, sep='\n')



2022-12-01 00:00:00
2022-12-31 00:00:00


In [99]:
print(socket)

NameError: name 'socket' is not defined

In [112]:


min_query_date= '2022-09-01'   
def start_end_month(date:datetime):
    #print(beg)
    # startTime = time.time()
    
    # Yearmonth = beg.strftime("%Y%m")
    # Year = beg.strftime("%Y")
    # Month = beg.strftime("%m")
    #sample_date =  datetime(int(Year), int(Month), 1)
    first_day = beg + relativedelta(day=1)
    last_day = (beg.replace(day=1) + datetime.timedelta(days=32)).replace(day=1)
    return first_day,last_day
for beg in pd.date_range(min_query_date,date.today() , freq='MS'): # date.today() date.today().strftime("%Y-%m-01")
    first_day_of_month, last_day_of_month = start_end_month(beg)
    print(first_day, last_day)

        #last_day = beg + relativedelta(month=1, day=1)
        #last_day = first_day + relativedelta(month=1)
    
                           


2022-09-01 00:00:00 2022-10-01 00:00:00
2022-10-01 00:00:00 2022-11-01 00:00:00
2022-11-01 00:00:00 2022-12-01 00:00:00
2022-12-01 00:00:00 2023-01-01 00:00:00
2023-01-01 00:00:00 2023-02-01 00:00:00
2023-02-01 00:00:00 2023-03-01 00:00:00


In [32]:
print(os.sep)
# /

print(os.sep is os.path.sep)
# True

\
True


In [31]:


import os
from pathlib import Path, PurePosixPath
# folder path
#print(LandingPath) 
Table_Folder = "ProductionRun/ProductionRun_hist/"+product+"/"
dir_path = os.path.join(LandingPath,Table_Folder.replace(f"/","\\")) #, LandingPath
    # path = Path(f"data/{color}/{dataset_file}.parquet")
#print(dir_path)    
    # df.to_parquet(path, compression="gzip")
    # path = PurePosixPath("data",color,f"{dataset_file}.parquet") # gcp path
# list to store files
def fast_scandir(dirname):
    subfolders= [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

# # Iterate directory
# for path in os.listdir(dir_path):
#     # check if current path is a file
#     if os.path.isfile(os.path.join(dir_path, path)):
#         res.append(path)
print(list(fast_scandir(dir_path)))

print("\nWe are listing out only the directories in current directory -")


['c:\\BI\\GitHub\\LearningSparkV2-master\\SuppliesML\\Data\\Dev\\Landing\\ProductionRun\\ProductionRun_hist\\IrisPlusSP\\2020', 'c:\\BI\\GitHub\\LearningSparkV2-master\\SuppliesML\\Data\\Dev\\Landing\\ProductionRun\\ProductionRun_hist\\IrisPlusSP\\2021', 'c:\\BI\\GitHub\\LearningSparkV2-master\\SuppliesML\\Data\\Dev\\Landing\\ProductionRun\\ProductionRun_hist\\IrisPlusSP\\2022', 'c:\\BI\\GitHub\\LearningSparkV2-master\\SuppliesML\\Data\\Dev\\Landing\\ProductionRun\\ProductionRun_hist\\IrisPlusSP\\2020\\202011', 'c:\\BI\\GitHub\\LearningSparkV2-master\\SuppliesML\\Data\\Dev\\Landing\\ProductionRun\\ProductionRun_hist\\IrisPlusSP\\2020\\202012', 'c:\\BI\\GitHub\\LearningSparkV2-master\\SuppliesML\\Data\\Dev\\Landing\\ProductionRun\\ProductionRun_hist\\IrisPlusSP\\2021\\202101', 'c:\\BI\\GitHub\\LearningSparkV2-master\\SuppliesML\\Data\\Dev\\Landing\\ProductionRun\\ProductionRun_hist\\IrisPlusSP\\2021\\202102', 'c:\\BI\\GitHub\\LearningSparkV2-master\\SuppliesML\\Data\\Dev\\Landing\\Produ

In [ ]:
from calendar import monthrange
from datetime import datetime



firstdayofmonth = current.replace(day=1)
endmonth = monthrange(current.year, current.month)
lastdayofmonth = datetime(current.year, current.month, endmonth[1])

In [17]:
### 5.1.2.1 read & write & Test Count / Show() all space BI yearmonth file to one parquet file
Parameters_run_level_dfs = spark.read.format("parquet").load(LandingPath + "/ProductionRun/ProductionRun_hist/Timna2/2022/202201/*") # try to read empty



In [18]:
Parameters_run_level_dfs.schema.names

['Product',
 'Machine',
 'FOLDER_PATH',
 'Product_category',
 'Product_eng_name',
 'Series',
 'Product_name_win',
 'Size_Flag',
 'Parameter_Name',
 'Batch',
 'SAMPLE_ID',
 'SAMPLE_Date',
 'SK_Sample_Date',
 'Parameter_Critical_Flag',
 'Is_Sample_Deleted_Flg',
 'SAMPLE_Mean',
 'SAMPLE_stdev',
 'SAMPLE_Minimum',
 'SAMPLE_Maximum',
 'SAMPLE_Median',
 'Spec_target',
 'SAMPLE_Size',
 'LSL',
 'USL',
 'SL_enabled',
 'CH_ID',
 'ETL_DATE']

In [123]:
# Parameters_run_level_dfs = spark.read.format("parquet").load(LandingPath + "/ProductionRun/ProductionRun_hist/*/*/*")
Parameters_run_level_dfs = spark.read.format("parquet").load(LandingPath + "/ProductionRun/ProductionRun_hist/*/*/*/*") # try to read empty


In [125]:
Parameters_run_level_dfs.createOrReplaceTempView("ld_Parameters_run_level_dfs")

In [127]:
ld_Parameters_run_level_dfs_new = spark.sql("select *, Year(SAMPLE_DATE)*100+Month(SAMPLE_DATE) As YearMonth from ld_Parameters_run_level_dfs")


In [ ]:
ld_Parameters_run_level_dfs_new.createOrReplaceTempView("ld_Parameters_run_level_dfs_new")

In [129]:
startTime = time.time()
ld_Parameters_run_level_dfs_new.coalesce(1).write.parquet(LandingPath + "/ProductionRun/ProductionRun_Full/ProductionRun_Full_YM.parquet",mode="overwrite") #e")#.saveAsTable("ld_blanket_lifespan_run_extended")# = spark.read.format("parquet").load(SourcePath + '/test')
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(round(executionTime,2)))
Parameters_run_level_dfs.count()

Execution time in seconds: 6.96


494670

#### 5.1.2.1 TEST Load from SpaceBI > production run jdbc



#### Load With pyodbc - Blankets Lifespan

In [77]:
import time
from datetime import datetime, timedelta,date
from pyspark.sql.types import StructType, IntegerType, StringType, LongType
import pandas as pd
from pyspark.sql.functions import col
from pathlib import Path, PurePosixPath

# Build Dynamic Query per yearmonth filter
def Loaddata(YearMonth,StartYear,querypath):
    
    #YearMonth ="201903"
    Year=YearMonth[0:4]
    with open(querypath) as f: query_file = f.read()
    sql_string = query_file.replace("{YearMonth}",str(YearMonth)).replace("{StartYear}",str(StartYear))
    dynamic_sql = f"""{sql_string}"""
    #print(dynamic_sql)
    #print(YearMonth)
    return dynamic_sql


convert_dict = {#'Blanket_Impact_RowID': str,
                'Optimized_Lifespan': int,
                #'Replacement_DateTime': str
                }
 


# Read Query from Source >  Return DATAFRAME per yearmonth filter
def pandas_read_df(query,conn):   
    startTime = time.time()
    Pareto_pdf = pd.read_sql_query(sql = query, con = conn)     
    print("after_read_sql_query")
    
    Pareto_pdf = Pareto_pdf.astype(convert_dict)
    #Pareto_pdf = Pareto_pdf.tz_localize(tz="Asia/Jerusalem", nonexistent=pd.Timedelta('1H'))
    Pareto_pdf['ETL_Date'] = pd.to_datetime(Pareto_pdf['ETL_Date']) \
.dt.tz_localize('Asia/Jerusalem',nonexistent=pd.Timedelta('1H'))
    print("after_convert_in_pandas")
    Dataframe.to_csv("test4.csv",index= False)
    return Pareto_pdf


def WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,YearMonth):
    #Dataframe = Dateframe['Blanket_Impact_RowID'].astype('int')
    spark_read_df_temp =spark.createDataFrame(Dataframe,Blanket_lifespan_installed_base_Schema)
    #print(LandingPath)

    #New_era_details_dfs.coalesce(2).write.parquet(LandingPath + "/New_Era/New_Era_Run_Summary_N",mode="overwrite")
    
  
    path = os.path.join(LandingPath,Table_Folder,str(Year),str(YearMonth),"x")
    #path = PurePosixPath("c:/BI/GitHub/LearningSparkV2-master/SuppliesML/Data/Dev/Landing/Blanket_lifespan/2022/202212/c.parquet")
    # c:\BI\GitHub\LearningSparkV2-master\SuppliesML\Data\Dev\Landing\Blanket_lifespan\2022\202212\
    print(path)
    spark_read_df_temp.write.format("parquet")\
                .mode("overwrite")\
                .save("v.parquet")
    #spark_read_df_temp.coalesce(8).write.parquet(path=path,mode="overwrite") 
    executionTime = (time.time() - startTime)

    
Zone = LandingPath
#Table_Folder = "/Blanket_lifespan/Blanket_lifespan_Hist_New/"
Table_Folder = "Blanket_lifespan"
Table = "Blanketslifespan"
QueryPath = "IngestQueries/Blanketslifespan.sql"
#Conn = odbc_Hubble_ITG
StartYear = "2019"
startPipelineTime =  time.time()
for beg in pd.date_range('2022-12-01','2022-12-06', freq='MS'): # date.today()
    startTime = time.time()
    YearMonth = beg.strftime("%Y%m")
    Year=YearMonth[0:4]                                     
    dynamic_sql = Loaddata(YearMonth,StartYear,QueryPath)  
    #print(dynamic_sql)
    Dataframe = pandas_read_df(dynamic_sql,odbc_Hubble_ITG  ) #sql_native_Hubble_ITG
    path = os.path.join(LandingPath,Table_Folder,str(Year),str(YearMonth),"test")
    print(path)
    Dataframe.to_parquet(f"{path}.parquet")
    #os.makedirs(os.path.dirname(LandingPath + Table_Folder +"/"+str(YearMonth)+"/"), exist_ok=True)
    #path = os.path.join(LandingPath,Table_Folder,str(Year),str(YearMonth),"")
    #Dataframe.to_parquet(f"{path}")#LandingPath + Table_Folder +"/"+str(YearMonth)+"/")
    #WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,YearMonth)
    executionTime = (time.time() - startTime)
    print(f'Execution time in seconds for: Table '+Table+' > YearMonth: '+ YearMonth +"  is: "+ str(round(executionTime,2))+ "   numberofrow:  " + str(len(Dataframe.index)))
    
executionTime = (time.time() - startPipelineTime)
print(f'{datetime.now()} total Execution time in seconds for: Table '+Table+' > all yearmonth is: '+ str(round(executionTime,2))) 




after_read_sql_query
after_convert_in_pandas
c:\BI\GitHub\LearningSparkV2-master\SuppliesML\Data\Dev\Landing\Blanket_lifespan\2022\202212\test
Execution time in seconds for: Table Blanketslifespan > YearMonth: 202212  is: 0.32   numberofrow:  10
2023-02-16 20:30:29.329376 total Execution time in seconds for: Table Blanketslifespan > all yearmonth is: 0.32


In [57]:
Table_Folder = "Blanket_lifespan"
path = os.path.join(LandingPath,Table_Folder,str(Year),str(YearMonth),"")
print(path)

print(os.sep)
# /

print(os.sep is os.path.sep)
# True

c:\BI\GitHub\LearningSparkV2-master\SuppliesML\Data\Dev\Landing\Blanket_lifespan\2022\202212\
\
True


In [114]:
Blanketslifespan = spark.read.format("parquet").load(LandingPath + Table_Folder +"/*")

In [133]:
Blanketslifespan.coalesce(1).write.parquet(LandingPath + "/Blanket_lifespan/Blanket_lifespan_hist_PM.parquet",mode="overwrite")